# Task for Today  

***

## McDonald's Menu Item Type Prediction  

Given *data about McDonald's menu items*, let's try to predict the **type** of a given item.  
  
We will use a TensorFlow ANN with two inputs to make our predictions.

# Getting Started

In [ ]:
import numpy as np
import pandas as pd

import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf

In [ ]:
data = pd.read_csv('../input/nutrition-facts/menu.csv')

In [ ]:
data

In [ ]:
data.info()

# Encoding Label Column

In [ ]:
data['Category'].unique()

In [ ]:
label_encoder = LabelEncoder()

data['Category'] = label_encoder.fit_transform(data['Category'])

label_mappings = dict(enumerate(label_encoder.classes_))
label_mappings

# Encoding Item Column

In [ ]:
data

In [ ]:
names = data['Item'].copy()

tokenizer = Tokenizer()
tokenizer.fit_on_texts(names)

names = tokenizer.texts_to_sequences(names)

In [ ]:
vocab_length = len(tokenizer.word_index) + 1

max_seq_length = np.max(list(map(lambda x: len(x), names)))

print("Vocabulary length:", vocab_length)
print("Max sequence length:", max_seq_length)

In [ ]:
names = pad_sequences(names, maxlen=max_seq_length, padding='post')
names

In [ ]:
data = data.drop('Item', axis=1)

In [ ]:
data

# Cleaning Serving Size Feature

In [ ]:
data['Serving Size']

In [ ]:
data['Serving Size'].unique()

In [ ]:
units = []

def get_grams(serving):
    units.append(0)
    return np.float(re.search(r'(?<=\()[\d]+', serving).group(0))

def get_ml(serving):
    units.append(1)
    return np.float(re.search(r'(?<=\()[\d]+', serving).group(0))

def get_fl_oz(serving):
    units.append(2)
    return np.float(re.search(r'^[\d.]+', serving).group(0))

In [ ]:
def get_units(serving):
    if ' g)' in serving:
        return get_grams(serving)
    
    elif ' ml)' in serving:
        return get_ml(serving)
    
    else:
        return get_fl_oz(serving)

In [ ]:
data['Serving Size'] = data['Serving Size'].apply(get_units)
data['Serving Units'] = units

In [ ]:
def onehot_encode(df, column, prefix):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=prefix)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [ ]:
data = onehot_encode(data, 'Serving Units', 'units')

# Splitting/Scaling

In [ ]:
data

In [ ]:
y = data['Category'].copy()
X = data.drop('Category', axis=1).copy()

In [ ]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [ ]:
names_train, names_test, X_train, X_test, y_train, y_test = train_test_split(names, X, y, train_size=0.7)

# Modeling

In [ ]:
num_classes = len(y.unique())
print("Number of classes:", num_classes)

name_feature_length = names.shape[1]
print("Name feature length:", name_feature_length)

other_feature_length = X.shape[1]
print("Other feature length:", other_feature_length)

In [ ]:
# Name features
name_input = tf.keras.Input(shape=(name_feature_length,), name="name_input")

name_embedding = tf.keras.layers.Embedding(
    input_dim=vocab_length,
    output_dim=64,
    input_length=name_feature_length,
    name="name_embedding"
)(name_input)

name_flatten = tf.keras.layers.Flatten(name="name_flatten")(name_embedding)


# Other features
other_input = tf.keras.Input(shape=(other_feature_length,), name="other_input")

dense_1 = tf.keras.layers.Dense(64, activation='relu', name="dense_1")(other_input)
dense_2 = tf.keras.layers.Dense(64, activation='relu', name="dense_2")(dense_1)


# Combined
concat = tf.keras.layers.concatenate([name_flatten, dense_2], name="concatenate")

outputs = tf.keras.layers.Dense(num_classes, activation='softmax', name="output_layer")(concat)


# Create model
model = tf.keras.Model(inputs=[name_input, other_input], outputs=outputs)

In [ ]:
print(model.summary())
tf.keras.utils.plot_model(model)

# Training

In [ ]:
batch_size = 32
epochs = 100

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    [names_train, X_train],
    y_train,
    validation_split=0.2,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau()
    ]
)

# Results

In [ ]:
results = model.evaluate([names_test, X_test], y_test, verbose=0)

print("Model Accuracy:", results[1])

# Data Every Day  

This notebook is featured on Data Every Day, a YouTube series where I train models on a new dataset each day.  

***

Check it out!  
https://youtu.be/YFvFoTZLKlA